<a href="https://colab.research.google.com/github/dzastin96/product-category-classifier/blob/main/notebooks/product_category_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📝 Product Data Preparation
### Author: Dzastin Januzi

## 🎯 Goal
The goal of this notebook is to **clean, standardize, engineer, and audit the product dataset** to ensure it is ready for analysis and modeling.  

Key objectives include:
- 🏷️ Standardizing column names for consistency  
- 📅 Converting column types where needed (e.g., `listing_date` → datetime, `category_label` → category)  
- 🔢 Rounding numeric metrics to two decimals where appropriate (e.g., `number_of_views`, `merchant_rating`)  
- 🧹 Identifying and removing missing values and duplicates  
- 📊 Creating engineered features:
  - `days_since_listing` → time since product listing (base for time‑normalized features)  
  - `views_per_day` → popularity normalized by time since listing  
  - `popularity_score` → popularity adjusted by merchant quality  
- 📑 Producing an audit‑ready summary (`info`) and preview (`head`) of the final dataset
- 💾 Saving the cleaned and enriched DataFrame to `data/final_products_data.csv` for use in the modeling notebook


## 📑 Columns Description

### 🟢 Initial Columns (from raw DataFrame)

| Column             | Description                                                                 |
|--------------------|-----------------------------------------------------------------------------|
| 🆔 product ID      | Unique identifier for each product (int64)                                  |
| 🏷️ Product Title   | Name/title of the product (object)                                          |
| 🏪 Merchant ID     | Unique identifier for the merchant (int64)                                  |
| 📂 Category Label  | Category under which the product is listed (object)                         |
| 🔢 _Product Code   | Internal product code (object)            |
| 👁️ Number_of_Views | Number of times the product listing has been viewed (float64)               |
| ⭐ Merchant Rating | Rating score of the merchant (float64, typically 1–5 scale)                 |
| 📅 Listing Date    | Date when the product was listed (object, raw string before conversion)     |

## 📥 1. Load Data & Preview

We begin by loading the raw product dataset and inspecting the first few rows.  
This helps verify the structure, column names, and initial data quality.

In [29]:
import pandas as pd
from IPython.display import display

# Load the CSV file into a DataFrame from data folder
df = pd.read_csv('../data/IMLP4_TASK_03-products.csv')

# Display the first 5 rows of the DataFrame
display(df.head(5).style.set_caption("FIRST 5 R0WS"))

,product ID,Product Title,Merchant ID,Category Label,_Product Code,Number_of_Views,Merchant Rating,Listing Date
0,1,apple iphone 8 plus 64gb silver,1,Mobile Phones,QA-2276-XC,860.000000,2.500000,5/10/2024
1,2,apple iphone 8 plus 64 gb spacegrau,2,Mobile Phones,KA-2501-QO,3772.000000,4.800000,12/31/2024
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold,3,Mobile Phones,FP-8086-IE,3092.000000,3.900000,11/10/2024
3,4,apple iphone 8 plus 64gb space grey,4,Mobile Phones,YI-0086-US,466.000000,3.400000,5/2/2022
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free,5,Mobile Phones,NZ-3586-WP,4426.000000,1.600000,4/12/2023


## 🔍 2. Initial Audit

The purpose of this section is to **inspect the raw dataset** before any cleaning or transformation.  
We want to understand its structure, identify missing values, and check for duplicates.

### Steps:
- 📊 Display basic DataFrame information (`df.info()`)
- ⚠️ Count missing values per column
- 🔁 Check for duplicated rows

In [30]:
# Display the information of the DataFrame
df.info()

# Count missing values per column
df_nan_counts = df.isnull().sum().to_frame(name='Count')
display(df_nan_counts.style.set_caption("Missing Values per Column"))

# Count for duplicated products if exists
count_duplicated_rows = df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35311 entries, 0 to 35310
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   product ID       35311 non-null  int64  
 1   Product Title    35139 non-null  object 
 2   Merchant ID      35311 non-null  int64  
 3    Category Label  35267 non-null  object 
 4   _Product Code    35216 non-null  object 
 5   Number_of_Views  35297 non-null  float64
 6   Merchant Rating  35141 non-null  float64
 7    Listing Date    35252 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 2.2+ MB


,Count
product ID,0
Product Title,172
Merchant ID,0
Category Label,44
_Product Code,95
Number_of_Views,14
Merchant Rating,170
Listing Date,59


## 🧹 3. Cleaning & Transformation

In this section we apply transformations to standardize and clean the dataset.  
The goal is to ensure consistent column naming, correct data types, and removal of invalid rows.

### Steps:
- 📝 Standardize column names (lowercase, underscores, remove special characters)
- 📅 Convert `listing_date` to proper datetime format
- 🔢 Round numeric columns (`number_of_views`, `merchant_rating`) to 2 decimals
- ⚠️ Remove rows with missing values
- 🔁 Remove duplicated rows

In [ ]:
# Standardize column names
df.columns = (
    df.columns
    .str.strip()                            # remove leading/trailing spaces
    .str.lower()                            # convert to lowercase
    .str.replace(r'^_', '', regex=True)     # removes underscore only at start
    .str.replace(' ', '_')                  # replace spaces with underscores
    .str.replace(r'[^\w_]', '', regex=True) # remove special characters
)

# Convert 'category_label' to categorical type
df['category_label'] = df['category_label'].astype('category')

# Convert 'listing_date' to datetime format (assuming format is 'YYYY-MM-DD')
df['listing_date'] = pd.to_datetime(df['listing_date'], errors='coerce', dayfirst=False)

# Round 'number_of_views' and 'merchant_rating' to 2 decimal places (NOT WORKING FOR DISPLAY())
# display() has own formatting settings for floats and datetime, and does not reflect changes made to the DataFrame itself. But we will use display() for better visualization in Jupyter Notebooks.
df['number_of_views'] = df['number_of_views'].round(2)
df['merchant_rating'] = df['merchant_rating'].round(2)


# Filter out rows that contain at least one missing value
rows_with_nan = df[df.isnull().any(axis=1)]

# Count the number of rows before removing missing values
rows_before = len(df)

# Remove rows with any missing values
df = df.dropna()

# Count the number of rows after removing missing values
rows_after = len(df)

## 📊 4. Statistics & Preview

In this section we summarize the results of the cleaning process and preview the cleaned dataset.  
This provides a clear before/after comparison and confirms that the dataset is ready for analysis.

In [32]:
# Print the statistics
print("\n=== Prepare Data Statistics ===")
print("\n✅ Column names have been standardized.")
print("✅ Convert 'category_label' to category type.")
print("✅ 'listing_date' column has been converted to datetime format.")
print("✅ Numeric metrics rounded to 2 decimals.")
if count_duplicated_rows > 0:
    df = df.drop_duplicates() # keep first occurrence
    print(f"✅ Number of duplicated rows removed: {count_duplicated_rows}.")
else:
    print("ℹ️ No duplicated rows found.")

print(f"✅ Number of rows before removing missing values: {rows_before}")
print(f"✅ Number of rows with missing values: {len(rows_with_nan)}")
print(f"✅ Number of rows after removing missing values: {rows_after}")

# Display the first 5 rows of the DataFrame
display(df.head(5).style.set_caption("FIRST 5 R0WS"))



=== Prepare Data Statistics ===

✅ Column names have been standardized.
✅ Convert 'category_label' to category type.
✅ 'listing_date' column has been converted to datetime format.
✅ Numeric metrics rounded to 2 decimals.
ℹ️ No duplicated rows found.
✅ Number of rows before removing missing values: 35311
✅ Number of rows with missing values: 551
✅ Number of rows after removing missing values: 34760


,product_id,product_title,merchant_id,category_label,product_code,number_of_views,merchant_rating,listing_date
0,1,apple iphone 8 plus 64gb silver,1,Mobile Phones,QA-2276-XC,860.000000,2.500000,2024-05-10 00:00:00
1,2,apple iphone 8 plus 64 gb spacegrau,2,Mobile Phones,KA-2501-QO,3772.000000,4.800000,2024-12-31 00:00:00
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold,3,Mobile Phones,FP-8086-IE,3092.000000,3.900000,2024-11-10 00:00:00
3,4,apple iphone 8 plus 64gb space grey,4,Mobile Phones,YI-0086-US,466.000000,3.400000,2022-05-02 00:00:00
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free,5,Mobile Phones,NZ-3586-WP,4426.000000,1.600000,2023-04-12 00:00:00


## 🧩 5. Feature Engineering

In this step we create new features that combine existing signals to improve model performance.  
Each feature is designed with a clear purpose and documented for transparency.

### Features:
- 📅 **days_since_listing** 
    → Derived from `df['listing_date']`, measures how many days have passed since product listing.
    → Provides a time dimension that helps normalize popularity metrics and detect recency effects (new vs. old listings). 
- 📊 **views_per_day** 
    → Captures product popularity normalized by `days_since_listing` (time since listing).
    → Prevents bias toward older listings with more accumulated views, ensuring fair comparison across products.
- ⭐ **popularity_score** 
    → Adjusts product popularity by merchant quality.
    → Balances raw popularity with seller credibility, highlighting products that are both popular and trustworthy.


In [33]:
from datetime import datetime

# Days since listing
df['days_since_listing'] = (datetime.today() - df['listing_date']).dt.days
print("✅ Created 'days_since_listing' feature.")

# Views per day (popularity normalized by time). If days_since_listing is 0, we avoid division by zero by replacing it with 1.
df['views_per_day'] = (df['number_of_views'] / df['days_since_listing'].replace(0, 1)).round(2)
print("✅ Created 'views_per_day' feature.")

# Popularity_score views (popularity adjusted by merchant rating)
df['popularity_score'] = (df['number_of_views'] * df['merchant_rating']).round(2)
print("✅ Created 'popularity_score' feature.")

✅ Created 'days_since_listing' feature.
✅ Created 'views_per_day' feature.
✅ Created 'popularity_score' feature.


## 📑 6. Final Data Frame

This step provides a final overview of the cleaned and feature‑engineered dataset.  
It serves as a quick resume of all transformations applied and confirms readiness for modeling.

### Actions:
- 📋 Display DataFrame info (column names, types, non‑null counts)
- 👀 Preview the first 5 rows of the final dataset

In [34]:
print("📋 Final DataFrame Info:")
df.info()

print("\n👀 Preview of Final DataFrame:")
display(df.head(5).style.set_caption("📑 FIRST 5 ROWS (Final DataFrame)"))

📋 Final DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 34760 entries, 0 to 35310
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   product_id          34760 non-null  int64         
 1   product_title       34760 non-null  object        
 2   merchant_id         34760 non-null  int64         
 3   category_label      34760 non-null  category      
 4   product_code        34760 non-null  object        
 5   number_of_views     34760 non-null  float64       
 6   merchant_rating     34760 non-null  float64       
 7   listing_date        34760 non-null  datetime64[ns]
 8   days_since_listing  34760 non-null  int64         
 9   views_per_day       34760 non-null  float64       
 10  popularity_score    34760 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 3.0+ MB

👀 Preview of Final DataFrame:


,product_id,product_title,merchant_id,category_label,product_code,number_of_views,merchant_rating,listing_date,days_since_listing,views_per_day,popularity_score
0,1,apple iphone 8 plus 64gb silver,1,Mobile Phones,QA-2276-XC,860.000000,2.500000,2024-05-10 00:00:00,561,1.530000,2150.000000
1,2,apple iphone 8 plus 64 gb spacegrau,2,Mobile Phones,KA-2501-QO,3772.000000,4.800000,2024-12-31 00:00:00,326,11.570000,18105.600000
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim free smartphone in gold,3,Mobile Phones,FP-8086-IE,3092.000000,3.900000,2024-11-10 00:00:00,377,8.200000,12058.800000
3,4,apple iphone 8 plus 64gb space grey,4,Mobile Phones,YI-0086-US,466.000000,3.400000,2022-05-02 00:00:00,1300,0.360000,1584.400000
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked sim free,5,Mobile Phones,NZ-3586-WP,4426.000000,1.600000,2023-04-12 00:00:00,955,4.630000,7081.600000


### 📑 Final Columns Description (After Cleaning & Feature Engineering)

| Column               | Description                                                                                   |
|----------------------|-----------------------------------------------------------------------------------------------|
| 🆔 product_id        | Unique identifier for each product (int64)                                                    |
| 🏷️ product_title    | Name/title of the product (object)                                                            |
| 🏪 merchant_id       | Unique identifier for the merchant (int64)                                                    |
| 📂 category          | Product category (category)                                                                  | 
| 🔢 product_code      | Internal product code (object)                                                               |
| 👁️ number_of_views  | Number of times the product listing has been viewed (float64)                                 |
| ⭐ merchant_rating   | Rating score of the merchant (float64, 1–5 scale)                                            |
| 📅 listing_date      | Date when the product was listed (datetime64)                                                |
| ⏳ days_since_listing| Time since product listing (int64)                                                            |
| 📊 views_per_day     | Average number of views per day since listing (float64)                                       |
| ⭐ popularity_score  | Balances popularity with seller credibility (float64)                                         |

## 💾 7. Save Cleaned & Engineered DataFrame

We save the final version of the dataset — fully cleaned, standardized, and enriched with engineered features — to the `data/` folder.  
This file will be used as the input for our next notebook, where we build and train a product category classification model.

Saved file: `data/final_products_data.csv`

In [35]:
df.to_csv("../data/final_product_data.csv", index=False)
print("✅ Final DataFrame saved to 'data/final_products_data.csv'")

✅ Final DataFrame saved to 'data/final_products_data.csv'
